
# Brazilian Airline Historical Series Analysis

#### Frederico Horst

### Data Sources:
- Historical air fares by origin, destination and airline: available at [ANAC website](https://sistemas.anac.gov.br/sas/downloads/view/frmDownload.aspx)
- Inflation data, using IPCA index: available at [IBGE website](https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?=&t=series-historicas)
- More information on air fares on [ANAC website](https://www.anac.gov.br/assuntos/dados-e-estatisticas/mercado-do-transporte-aereo)

### Goals:
- Build a database for historical deflated prices.
- Calculate the confidence interval for the average price range by route, considering a 95% confidence.
- Confidence intervals will be calculated by route, not considering airline differences. We want to take a closer look to the consumer point of view.


In [1]:
# external libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession, SQLContext

# internal lib:
import files_processor

# spark configs:
spark = SparkSession.builder \
        .master("local") \
        .appName("anac-prices") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

sqlContext = SQLContext(spark)

In [2]:
# importing and cleaning files
anac_table = files_processor.files_cleaning(
    path_source='csv_files_from_anac',
    inflation_file='ipca_historico.csv')

###########################################################################
BEGGINING DATA CLEANING PROCESS
importing IPCA file
Inflation series imported successfully
#########################
beginning data cleaning
THIS IS THE END
###########################################################################


In [3]:
anac_table.registerTempTable('anac_table')
anac_table.show()

+----+-----+----------+-------+------+-----------+---------+------+-----+------------------+
|year|month|year_month|company|origin|destination|    route|tariff|seats|   deflated_tariff|
+----+-----+----------+-------+------+-----------+---------+------+-----+------------------+
|2015|   11|    201511|    AZU|  SBPA|       SBSL|SBPA>SBSL| 985.9|    5|22.152816007370042|
|2015|   11|    201511|    AZU|  SBGR|       SBCA|SBGR>SBCA| 305.9|    1| 6.873462234156097|
|2015|   11|    201511|    AZU|  SBPJ|       SBPA|SBPJ>SBPA|187.39|    2| 4.210585446415531|
|2015|   11|    201511|    AZU|  SBCT|       SBSP|SBCT>SBSP|100.01|   13| 2.247188486557539|
|2015|   11|    201511|    AZU|  SBPA|       SBRP|SBPA>SBRP| 833.9|    1| 18.73743104629869|
|2015|   11|    201511|    AZU|  SBFZ|       SBMO|SBFZ>SBMO| 603.9|    1|13.569415206608321|
|2015|   11|    201511|    AZU|  SBGL|       SBMO|SBGL>SBMO| 425.9|    5| 9.569818782370323|
|2015|   11|    201511|    AZU|  SBGR|       SBRF|SBGR>SBRF| 500.0|  1

In [8]:
# importing clean airports names
# airports = sqlContext.read.csv('aeroportos.csv', sep=";", inferSchema="true", header="true")
# airports.registerTempTable('airports')

# calculating tariff mean without confidence interval
grouping_query = """
    SELECT DISTINCT 
        year_month,
        route,
        SUM(seats) OVER partial AS sold_seats,
        SUM(deflated_tariff) OVER partial AS total_deflated_tariff,
        SUM(tariff) OVER partial AS total_tariff,
        (SUM(deflated_tariff) OVER partial * SUM(seats) OVER partial) / SUM(seats) OVER total AS deflated_tariff_mean,
        (SUM(tariff) OVER partial * SUM(seats) OVER partial) / SUM(seats) OVER total AS tariff_mean

    FROM anac_table

    WINDOW total AS (PARTITION BY year_month),
        partial AS (PARTITION BY year_month, route)
    
    """

anac_sts = sqlContext.sql(grouping_query)



In [9]:
# anac_sts_df = anac_sts.toPandas()
# route = anac_sts_df.route.unique()
# anac_sts_df.head()

anac_sts.show()

+----------+---------+----------+---------------------+------------+--------------------+------------------+
|year_month|    route|sold_seats|total_deflated_tariff|total_tariff|deflated_tariff_mean|       tariff_mean|
+----------+---------+----------+---------------------+------------+--------------------+------------------+
|    200206|SBGL>SBPA|     10905|    818.4799734193633|     15273.0|   7.561658087523834| 141.1020522448122|
|    200206|SBPA>SBSP|     36766|    512.6418795082583|      9566.0|  15.967751817657085|297.96144246784473|
|    200206|SBUL>SBSP|      4418|   291.31520562480574|      5436.0|  1.0903656818735814|20.346441696897404|
|    200206|SBCT>SBPA|      9930|    522.1273083889776|      9743.0|   4.392471633631049| 81.96439917788211|
|    200206|SBSV>SBGR|     17808|    1044.361796765308|     19488.0|  15.756125538008217| 294.0124537643409|
|    200206|SBBH>SBCT|      4108|     395.922873281101|      7388.0|  1.3779210545193294| 25.71228246154159|
|    200206|SBSP>SB

In [ ]:
# https://seaborn.pydata.org/examples/jitter_stripplot.html